In [1]:
import glob
import pandas as pd
from tensorflow import keras
import numpy as np
import os 
import matplotlib.pylab as plt
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.layers import TimeDistributed, Conv2D, Conv2DTranspose, MaxPooling2D, AveragePooling2D, BatchNormalization, concatenate, Input, ConvLSTM2D, Reshape, Conv3D, Flatten, LSTM, GRU, Dense,Dropout, Add
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.models import Sequential, load_model
from sklearn.utils import shuffle

In [2]:
# mean and standard deviation for train data 
mu = 13.262550318358528
std = 36.12859290913875

In [3]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, batch_size=32, dim=(120,120), n_channels=1, n_timesteps = 4, shuffle=True, augment_data = True,
                standardize = False):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_timesteps = n_timesteps 
        self.shuffle = shuffle
        self.augment_data = augment_data  
        self.standardize = standardize 
        self.on_epoch_end() 

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' 
        
        if self.augment_data == True:  # only augment data when training - ignore this for now 
            # Initialization
            X = np.empty((self.batch_size*6, 120, 120, 4))
            y = np.empty((self.batch_size*6, 120, 120, 1)) 

            # Generate data
            for i, ID in enumerate(list_IDs_temp):
                data = np.load('./storage/precipitation/train/' + ID)
                # Store sample
                x_data = data[:,:,:4] 
                y_data = data[:,:,-1].reshape((120,120,1)) 
                
                X[i,] = x_data
                y[i] = y_data 
                
                # add 90 degrees rotation 
                X[i+self.batch_size,] = np.rot90(x_data)
                y[i+self.batch_size] = np.rot90(y_data)  
                
                # add 180 degrees rotation 
                X[i+self.batch_size*2,] = np.rot90(np.rot90(x_data)) 
                y[i+self.batch_size*2] = np.rot90(np.rot90(y_data)) 
                
                # add 270 degrees rotation 
                X[i+self.batch_size*3,] = np.rot90(np.rot90(np.rot90(x_data)))
                y[i+self.batch_size*3] = np.rot90(np.rot90(np.rot90(y_data)))  
                
                # add horizontal flip 
                X[i+self.batch_size*4,] = np.fliplr(x_data)
                y[i+self.batch_size*4] = np.fliplr(y_data) 
                
                # add vertical filp 
                X[i+self.batch_size*5,] = np.flipud(x_data) 
                y[i+self.batch_size*5] = np.flipud(y_data)
            
            # shuffle once more to make training harder 
            X,y = shuffle(X,y) 
            return (X, y)
        
        else: 
            # Initialization
            size = 20 
            #X = np.empty((self.batch_size * 36, 4, size, size, 1))
            #y = np.empty((self.batch_size * 36, size, size, 1)) 

            x_train = [] 
            y_train = [] 
            
            # Generate data
            for i, ID in enumerate(list_IDs_temp):
                data = np.load('./storage/precipitation/train/' + ID).astype(np.float32) 
                if self.standardize:  
                    data = (data - mu)/std

                for j in range(6): 
                    for k in range(6):  
                        x_train.append(data[size*j:size*(j+1),size*k:size*(k+1),:4]) 
                        y_train.append(data[size*j:size*(j+1),size*k:size*(k+1),-1]) 
                        
            x_train = np.asarray(x_train).reshape((-1,4,20,20,1)) 
            y_train = np.asarray(y_train).reshape((-1,20,20,1))
        
            return x_train, y_train 


In [8]:
def simple_rnn(): 
    inputs = Input((4,20,20,1)) 
    bn = BatchNormalization()(inputs)
    conv = ConvLSTM2D(32, 3, padding = 'same', return_sequences = True)(bn)
    bn = BatchNormalization()(conv) 
    conv = ConvLSTM2D(32, 3, padding = 'same', return_sequences = False)(bn)
    bn = BatchNormalization()(conv)  
    outputs = Conv2D(1, 1, padding = "same", activation = 'relu')(bn) 
    model = Model(inputs=inputs,outputs=outputs) 
    model.compile(loss='mae',optimizer='adam')
    return model 


In [9]:
model = simple_rnn() 
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 4, 20, 20, 1)]    0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 4, 20, 20, 1)      4         
_________________________________________________________________
conv_lst_m2d_6 (ConvLSTM2D)  (None, 4, 20, 20, 32)     38144     
_________________________________________________________________
batch_normalization_7 (Batch (None, 4, 20, 20, 32)     128       
_________________________________________________________________
conv_lst_m2d_7 (ConvLSTM2D)  (None, 20, 20, 32)        73856     
_________________________________________________________________
batch_normalization_8 (Batch (None, 20, 20, 32)        128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 20, 20, 1)         33  

In [ ]:
# due to time limitations, we will not do k-fold ensemble 
# fix the train and validation sets. 
train_files = [x for x in os.listdir('./storage/precipitation/train/')] 
train_files = shuffle(train_files)
k = int(0.8 * len(train_files)) 
train_data = train_files[:k]
val_data = train_files[k:]

partition = {'train':[], 'validation':[]} 

for filename in train_data: 
    partition['train'].append(filename) 
for filename in val_data: 
    partition['validation'].append(filename)  

params_train_gen = {'dim': (120,120),
                    'batch_size': 32,
                    'n_channels': 4,
                    'n_timesteps': 4,
                    'shuffle': True,
                    'augment_data': False}  

params_val_gen = {'dim': (120,120), 
                  'batch_size': 32, 
                  'n_channels': 4, 
                  'n_timesteps': 4,
                  'shuffle': True,
                  'augment_data': False} 
         

training_generator = DataGenerator(partition['train'], **params_train_gen)
validation_generator = DataGenerator(partition['validation'], **params_val_gen) 
model = simple_rnn()
        
model_path = './storage/precip_rnn_whole_2/epoch_{epoch:03d}_val_loss_{val_loss:.3f}.h5'
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_loss', patience = 3, verbose = 1, factor = 0.8)
checkpoint = ModelCheckpoint(filepath = model_path, monitor = 'val_loss', verbose = 1, save_best_only = True)
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 16) 
history = model.fit_generator(generator = training_generator, validation_data = validation_generator, epochs = 300, callbacks = [checkpoint, early_stopping, learning_rate_reduction])


Epoch 1/300
1567/1568 [============================>.] - ETA: 0s - loss: 7.4555
Epoch 00001: val_loss improved from inf to 6.53018, saving model to ./storage/precip_rnn_whole_2/epoch_001_val_loss_6.530.h5
1568/1568 [==============================] - 1230s 784ms/step - loss: 7.4543 - val_loss: 6.5302
Epoch 2/300
1567/1568 [============================>.] - ETA: 0s - loss: 6.0614
Epoch 00002: val_loss improved from 6.53018 to 5.78818, saving model to ./storage/precip_rnn_whole_2/epoch_002_val_loss_5.788.h5
1568/1568 [==============================] - 1223s 780ms/step - loss: 6.0611 - val_loss: 5.7882
Epoch 3/300
1567/1568 [============================>.] - ETA: 0s - loss: 5.7970
Epoch 00003: val_loss improved from 5.78818 to 5.69138, saving model to ./storage/precip_rnn_whole_2/epoch_003_val_loss_5.691.h5
1568/1568 [==============================] - 1223s 780ms/step - loss: 5.7973 - val_loss: 5.6914
Epoch 4/300
1567/1568 [============================>.] - ETA: 0s - loss: 5.6642
Epoch 000